In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy import *

Data Loading and Preprocessing

In [ ]:
df = pd.read_csv(r"E:\Study\Internship Drive\Prof Khare\Banking Dataset\bank-full.csv",delimiter = ';')  #Add your own path here.
y = df['y']
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [ ]:
#Dropping the columns based on attribute analysis done in WEKA
df = df.drop(columns = ['age','job','balance','day','campaign','loan','education','marital','default','contact','month','pdays','previous'])
df

,housing,duration,poutcome,y
0,yes,261,unknown,no
1,yes,151,unknown,no
2,yes,76,unknown,no
3,yes,92,unknown,no
4,no,198,unknown,no
...,...,...,...,...
45206,no,977,unknown,yes
45207,no,456,unknown,yes
45208,no,1127,success,yes
45209,no,508,unknown,no


For maximum accuracy, use duration poutcome housing attributes only. Observation based on attribute analysis done in WEKA.

Data Preprocessing over

In [ ]:
x = df.loc[:,df.columns!='y']
x_0 = x.loc[df['y']=='no',:]     #4000 rows
x_1 = x.loc[df['y']=='yes',:]    #521 rows
p_0 = len(x_0)/len(x)            #Class probability "no"
p_1 = len(x_1)/len(x)
m = x.shape[1]

In [ ]:
def classify (classid):
    dc_0 = dict()
    dc_1 = dict()
    count_0 = dict()
    count_1 = dict()
    
    for i in range(len(df)):
        vector = df.loc[i]
        class_value = vector[classid]
        if class_value not in dc_0:
            dc_0[class_value] = list()
            count_0[class_value] = 0
            dc_1[class_value] = list()
            count_1[class_value] = 0
        if vector.y == 'no':
            dc_0[class_value].append(vector)
            count_0[class_value] = count_0[class_value] + 1
        elif vector.y == 'yes':  
            dc_1[class_value].append(vector)
            count_1[class_value] = count_1[class_value] + 1
    #end for loop
    total_0 = len(x_0)
    total_1 = len(x_1)
    prob_0 = dict()
    prob_1 = dict()

    for i in count_0:
        if i not in prob_0:
            prob_0[i] = 0.0
        prob_0[i] = count_0[i] / total_0

    for i in count_1:
        if i not in prob_1:
            prob_1[i] = 0.0
        prob_1[i] = count_1[i] / total_1
    
    return dc_0,dc_1,count_0,count_1,prob_0,prob_1

In [ ]:
dc_0 = []
dc_1 = []
count_0 = []
count_1 = []
prob_0 = []
prob_1 = []

for i in range(m):
    dc0,dc1,c0,c1,p0,p1 = classify(i)
    dc_0.append(dc0)
    dc_1.append(dc1)
    count_0.append(c0)
    count_1.append(c1)
    prob_0.append(p0)
    prob_1.append(p1)

In [ ]:
df2 = pd.read_csv(r"E:\Study\Internship Drive\Prof Khare\Banking Dataset\bank.csv",delimiter = ';') #Add your own path for testing dataset.
#Dropping 
df2 = df2.drop(columns = ['age','job','balance','day','campaign','loan','education','marital','default','contact','month','pdays','previous'])
xtest = df2.loc[:,df2.columns!='y']
ytest = df2['y']
ytest

0       no
1       no
2       no
3       no
4       no
        ..
4516    no
4517    no
4518    no
4519    no
4520    no
Name: y, Length: 4521, dtype: object

In [ ]:
#Find pyes and pno for all training examples. Record your predictions.
prob_yes = []
prob_no = []
for j in range(len(xtest)):
    v = xtest.loc[j]
    pr = 1
    for i in range(m):
        class_value = v[i]
        pr = pr * prob_1[i][class_value]
    pyes = pr * p_1 * len(x)    
    
    pr = 1
    for i in range(m):
        class_value = v[i]
        pr = pr * prob_0[i][class_value]
    pno = pr * p_0 * len(x) 
    
    pyes = pyes /(pyes + pno)
    pno = pno / (pyes + pno)
    prob_yes.append(pyes)
    prob_no.append(pno)

In [ ]:
pred_probabilities = np.c_[prob_yes,prob_no]
prediction = []
no_correct = 0
no_wrong = 0
tp = 0
tn = 0
fp = 0
fn = 0

for i in range(len(df2)):
    if prob_yes[i]>prob_no[i]:
        prediction.append('yes')
    else:
        prediction.append('no')
        
    if prediction[i] == ytest[i]:
        if ytest[i]=='yes':#this is true positive
            tp = tp + 1
        else:
            tn = tn + 1
        no_correct = no_correct + 1
    else:
        if ytest[i]=='yes':
            fn = fn + 1
        else:
            fp = fp + 1
        no_wrong = no_wrong + 1
        #print(df.loc[i],prediction[i])

perc_accuracy = no_correct*100/(no_correct + no_wrong)
print(no_correct,no_wrong,perc_accuracy)

4110 411 90.9090909090909


In [ ]:
sensitivity = tp*100/(tp + fn)
specificity = tn*100/(tn + fp)
print('Sensitivity = ', sensitivity,'%')
print('Specificity = ', specificity,'%')

Sensitivity =  31.861804222648754 %
Specificity =  98.6 %
